In [1]:
#Importing all modules


import sys
import scripts_raw_data.fetch_raw_data as r
import scripts_raw_data.map_team_names as b
import scripts_data_process.build_allStats_from1980 as p
import scripts_data_process.build_splits as bs
import train_models as t
import hyperparameters_tuning.greedy_forward_feature_selection as gf
import hyperparameters_tuning.greedy_backward_feature_selection as gb
import hyperparameters_tuning.hyperparameters_tuner as ht



# I. Fetching the raw data.


In [2]:
r.main(1980, 2025)



[INFO] Downloading per-season raw data
[INFO] Downloading season 1980 from https://www.basketball-reference.com/leagues/NBA_1980_per_game.html...
[SKIP] Skipping season 1980, file already exists.
[INFO] Downloading season 1981 from https://www.basketball-reference.com/leagues/NBA_1981_per_game.html...
[SKIP] Skipping season 1981, file already exists.
[INFO] Downloading season 1982 from https://www.basketball-reference.com/leagues/NBA_1982_per_game.html...
[SKIP] Skipping season 1982, file already exists.
[INFO] Downloading season 1983 from https://www.basketball-reference.com/leagues/NBA_1983_per_game.html...
[SKIP] Skipping season 1983, file already exists.
[INFO] Downloading season 1984 from https://www.basketball-reference.com/leagues/NBA_1984_per_game.html...
[SKIP] Skipping season 1984, file already exists.
[INFO] Downloading season 1985 from https://www.basketball-reference.com/leagues/NBA_1985_per_game.html...
[SKIP] Skipping season 1985, file already exists.
[INFO] Downloadin

We build a mapping between the teams abbreviations and full team names. 

In [3]:
b.main()


[INFO] Building team abbreviation mapping
Scanning player CSVs for team IDs: 100%|██████████| 46/46 [00:00<00:00, 271.53it/s]
[WARN] Unknown team abbreviations not found on site: ['2TM', '3TM', '4TM', '5TM', 'BRK', 'CHH', 'CHO', 'KCK', 'NOK', 'NOP', 'SDC', 'SEA', 'VAN', 'WSB']
[ERROR] Run merge_team_mapping() to resolve them.
[DONE] Saved mapping to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\raw_data\team_id_to_name.csv

[INFO] Fetching missing teams from historical_teams.csv
[DONE] Merged mapping saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\raw_data\team_id_to_name.csv

[INFO] Checking for missing teams
Checking player team IDs: 100%|██████████| 46/46 [00:00<00:00, 279.42it/s]
[DONE] All team abbreviations in player stats are mapped.
Checking standings team names: 100%|██████████| 46/46 [00:00<00:00, 537.74it/s]
[DONE] All full team names in standings are mapped.

[INFO] Checking for duplicate team IDs
[DONE] No duplicate Team ID found in the mapping.

[INFO] Checking mapping
Va

# II. Data processing. 


We construct per season (year) player datasets, collecting all basic player statistics available since 1980 (e.g., G / GS / MP / FG / FGA / FG% / 3P / 3PA / 3P% / 2P / 2PA / 2P% / eFG%,
FT / FTA / FT% / ORB / DRB / TRB / STL / BLK / TOV / PF / AST / PTS), and enriching them with team context: team rank, win-loss differential, and one-hot position encoding. We prepare aligned inputs and labels (top-1, top-10 MVP) for downstream MVP prediction tasks. We build the following files : Data.csv (features per player), Y_top1.csv (binary label: MVP or not), Y_top10.csv (MVP-rank or -1), and Name.csv (player names), all aligned row-by-row. Each file is then stored in processed_data/allStats_from1980/year.

In [2]:
p.main(1980, 2025)

[INFO] Running pipeline from 1980 to 2025...

[INFO] Processing the raw data to per-season cleaned data
Creating per-season cleaned data: 100%|██████████| 46/46 [00:05<00:00,  8.98it/s]
[DONE] Saved per-season data to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\processed_data\allStats_from1980

[INFO] Filling missing values for per-season data
Imputing missing values with KNN: 100%|██████████| 46/46 [00:01<00:00, 25.55it/s]
[DONE] Finished filling missing values.

[INFO] Normalizing per-season data
Normalizing cleaned stats: 100%|██████████| 46/46 [00:01<00:00, 28.46it/s]
[DONE] Finished normalization.

[INFO] Verifying processed data integrity
Validating per-season files: 100%|██████████| 46/46 [00:00<00:00, 59.78it/s]
[DONE] Finished checking processed data integrity.


We build loso (leave one season out) splits from the processed data. The files are stored in datasets/allStats_from1980/year/train for train data (and datasets/allStats_from1980/year/test for test data). For one year, say 2025, the train files contains the data of all years except 2025 and the test file contains the data from 2025. 

In [3]:
bs.main(["all1980"], 1980, 2025)


[INFO] Creating LOSO splits for pipeline 'allStats_from1980' from 1980 to 2025...

[INFO] Creating leave-one-season-out splits for pipeline 'allStats_from1980'
Building LOSO splits: 100%|██████████| 46/46 [00:19<00:00,  2.40it/s]
[DONE] Finished creating LOSO splits for pipeline 'allStats_from1980'
[INFO] Checking LOSO splits for pipeline 'allStats_from1980' from 1980 to 2025...

[INFO] Checking LOSO splits integrity for pipeline 'allStats_from1980'
Checking splits: 100%|██████████| 46/46 [00:02<00:00, 22.11it/s]
[DONE] All LOSO splits are consistent for pipeline 'allStats_from1980'

[INFO] All selected pipelines finished.


# III. Training the models.

We chose to train the following models : logistic regression, random forest, and xgb.

In [5]:
models=["logreg", "xgb", "rf"]
for model in models:
    t.main(model=model)


{'pipeline': ['all1980'], 'model': 'logreg', 'start': 1980, 'end': 2025, 'verbose': False, 'full': False, 'datasets_base_dir': 'c:\\Users\\cleme\\Eloi\\Code\\HoML\\Projet-Homl\\datasets', 'models_base_dir': 'c:\\Users\\cleme\\Eloi\\Code\\HoML\\Projet-Homl\\models', 'vprint': <function main.<locals>.<lambda> at 0x000001A76B5B20E0>, 'model_class': <class 'sklearn.linear_model._logistic.LogisticRegression'>, 'model_name': 'LogisticRegression', 'pipelines_to_run': ['all1980'], 'p': 'all1980', 'pipeline_key': 'all1980', 'pipeline_name': 'allStats_from1980', 'pipeline_min_year': 1980, 'frame': <frame at 0x000001A768C84B40, file 'c:\\Users\\cleme\\Eloi\\Code\\HoML\\Projet-Homl\\train_models.py', line 502, code main>}
[INFO] Running model 'LogisticRegression' on pipeline 'allStats_from1980' from 1980 to 2025...
[INFO] Using hyperparameters: {'solver': 'saga', 'penalty': 'l2', 'class_weight': None, 'max_iter': 50000, 'C': 5.0}

[INFO] Training model for year 1980...
[DONE] Model saved to c:\Us

c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:44] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:44] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:44] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_1981.joblib

[INFO] Training model for year 1982...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_1982.joblib

[INFO] Training model for year 1983...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_1983.joblib

[INFO] Training model for year 1984...


c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:44] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:44] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:44] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_1984.joblib

[INFO] Training model for year 1985...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_1985.joblib

[INFO] Training model for year 1986...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_1986.joblib


...

[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_2019.joblib

[INFO] Training model for year 2020...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_2020.joblib

[INFO] Training model for year 2021...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_2021.joblib

[INFO] Training model for year 2022...


c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:47] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:47] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:48] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_2022.joblib

[INFO] Training model for year 2023...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_2023.joblib

[INFO] Training model for year 2024...
[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_2024.joblib

[INFO] Training model for year 2025...


c:\Users\cleme\miniconda3\envs\nba-mvp-prediction\lib\site-packages\xgboost\training.py:183: UserWarning: [18:17:48] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[DONE] Model saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\checkpoints\XGBClassifier_2025.joblib

[DONE] Summary saved to c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\models\XGBClassifier_allStats_from1980\summary_results.csv

[INFO] Aggregating global metrics over all years:

[SUMMARY] Recall@k (Top-k hit rate):
Top-1 hit rate: 0.826 (38/46)
Top-3 hit rate: 0.870 (40/46)
Top-5 hit rate: 0.913 (42/46)
Top-10 hit rate: 0.978 (45/46)

[SUMMARY] Avg Precision@k vs real top-k:
Precision@1 vs real top-1: 0.826
Precision@3 vs real top-3: 0.609
Precision@5 vs real top-5: 0.622
Precision@10 vs real top-10: 0.666

[SUMMARY] Avg Precision@k vs real top-10:
Precision@1 vs real top-10: 0.978
Precision@3 vs real top-10: 0.899
Precision@5 vs real top-10: 0.835
Precision@10 vs real top-10: 0.666

[SUMMARY] Avg Mean Absolute Rank Error@k:
Mean Abs Rank Error@1: 0.739
Mean Abs Rank Error@3: 2.188
Mean Abs Rank Error@5: 2.826
Mean Abs Rank Error@10: 2.857

[SUMMA

# IV. Optimization : feature selection and hyperparameter tuning. 


## A. Feature selection

We have implemented to ways of doing feature selection : forward greedy selection and backward greedy selection. We applied them both to logistic regression, random forest and xgb in order to find a better set of feature to train the models on.

In [3]:
#greedy forward feature selection 

gf.main(["all1980"],  "logreg", 2)

[INFO] Running greedy forward for pipeline: allStats_from1980
[INFO] Running greedy forward selection (Recall@1) on dataset: c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\datasets\allStats_from1980


[INFO] Added feature 1 | Total: 1 | Recall@1: 0.283


[INFO] Added feature 26 | Total: 2 | Recall@1: 0.587


[INFO] Added feature 25 | Total: 3 | Recall@1: 0.652


[INFO] Added feature 18 | Total: 4 | Recall@1: 0.674


KeyboardInterrupt: 

In [3]:
#greedy backward feature selection

gb.main(["all1980"],  "logreg", 2)

[INFO] Running greedy backward RFE for pipeline: allStats_from1980
[INFO] Running greedy backward RFE (Recall@1) on dataset: c:\Users\cleme\Eloi\Code\HoML\Projet-Homl\datasets\allStats_from1980


KeyboardInterrupt: 

## B. Hyperparameter Tuning.

We chose to tune the hyperparameters of the xgb, Logistic Regression and Random Forest, because these were the models that gave the best results when trained and evaluated. 

In [3]:
ht.main("logreg", "C", [1, 2, 3, 4, 5], combo={}, full=False)

[INFO] Using param values for 'C': [1, 2, 3, 4, 5]


#########################
## Pipeline: allStats_from1980
#########################


[INFO] Tuning 'C' on pipeline 'allStats_from1980' [1980-2025]


=== Testing C = 1 ===

C=1: 100%|██████████| 46/46 [00:42<00:00,  1.08it/s]

=== Testing C = 2 ===

C=2: 100%|██████████| 46/46 [00:47<00:00,  1.04s/it]

=== Testing C = 3 ===

C=3: 100%|██████████| 46/46 [00:57<00:00,  1.25s/it]

=== Testing C = 4 ===

C=4: 100%|██████████| 46/46 [00:52<00:00,  1.15s/it]

=== Testing C = 5 ===

C=5: 100%|██████████| 46/46 [01:13<00:00,  1.60s/it]

